In [1]:
# 구글 계정 액세스
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
tf.__version__

'2.8.0'

## 데이터 로드

In [3]:
import os, shutil
import matplotlib
import matplotlib.pyplot as plt
from os import remove
import numpy as np
import glob

In [ ]:
## 파일 확장자명 jpg로 통일
list_f = ['train_gochu', 'train_hobak', 'train_oe', 'train_ohobak']
list_i = ['jpeg', 'JPEG', 'JPG']
path = '/content/drive/MyDrive/Colab/img'

# for x in list_f:
#   path_ = (f'{path}/{x}')
for i in list_i:
  files = glob.glob(f'/content/drive/MyDrive/Colab/img/*/*.{i}')
  for name in files:
    if not os.path.isdir(name):
      src = os.path.splitext(name)
      os.rename(name, src[0]+'.jpg')

### 이미지, 라벨링 파일 전체 불러오기
- 파일 전처리 (잎 부위촬영 제외 모두 삭제)
- 숨김 파일 제외
- ???_과제번호(79)_질병 (0/1)_질병코드_작물코드_촬영부위코드(잎 03)_작물생육코드_해충피해코드(정상0 초기1 중기2 말기3)
- 질병코드 :
    - 정상 00

    - 고추 탄저병 01. => 작물코드 ( 01 )

    - 고추 흰가루병 02  => 작물코드 ( 01 )
    - 애호박 흰가루병 08 => 작물코드 ( 04 )
    - 오이 흰가루병 12 => 작물코드 ( 06 )
    - 호박 흰가루병 20 => 작물코드 ( 10 )

In [ ]:
# os.path.basename(파일경로)

In [15]:
## 숨긴파일, 라벨 파일 제외하고 불러오기
def load_image_path(path):
    file_list = os.listdir(path)
    print(file_list)
    file_list_py = [file for file in file_list if not file.startswith('.')]
    return file_list_py

In [16]:
for x in list_f:
  path_ = (f'{path}/{x}')
  img = load_image_path(path_)
  print(img)

['V006_79_1_02_01_03_13_3_0149z_20201028_56.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201028_55.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201028_59.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201028_58.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201028_51.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201103_17.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201103_2.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201103_19.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201103_13.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201103_16.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201103_14.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201103_20.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201103_1.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201103_18.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201103_10.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201103_4.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201110_8.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201103_22.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201117_1.jpg', 'V006_79_1_02_01_03_13_3_0149z_20201103_3.jpg', 'V006_79_1_02_01_03_13_3_

In [17]:
# 촬영부위가 잎이 아닌 이미지 모두 삭제
def leaf_img(datas): 
    ls = []
    for data in datas:
        if data.split('_')[5] != '03':
            ls.append(data)
    new_datas = [x for x in datas if x not in ls]
    return new_datas

In [18]:
leaf = leaf_img(img)
leaf

['V006_79_1_07_04_03_13_1_3033t_20201111_1516.jpg',
 'V006_79_1_07_04_03_13_1_3033t_20201111_1515.jpg',
 'V006_79_1_07_04_03_13_1_3033t_20201111_1520.jpg',
 'V006_79_1_07_04_03_13_1_3033t_20201111_1526.jpg',
 'V006_79_1_07_04_03_13_1_3033t_20201111_1511.jpg',
 'V006_79_1_07_04_03_13_1_3033t_20201111_1532.jpg',
 'V006_79_1_07_04_03_13_1_5950b_20201008_125.jpg',
 'V006_79_1_07_04_03_13_1_5950b_20201008_122.jpg',
 'V006_79_1_07_04_03_13_1_5950b_20200928_7.jpg',
 'V006_79_1_07_04_03_13_1_5950b_20201008_126.jpg',
 'V006_79_1_07_04_03_13_1_5950b_20200928_9.jpg',
 'V006_79_1_07_04_03_13_1_5950b_20200928_11.jpg',
 'V006_79_1_07_04_03_13_1_3033t_20201111_1528.jpg',
 'V006_79_1_07_04_03_13_1_5950b_20201008_124.jpg',
 'V006_79_1_07_04_03_13_1_3033t_20201111_1756.jpg',
 'V006_79_1_07_04_03_13_1_5950b_20200928_10.jpg',
 'V006_79_1_07_04_03_13_1_5950b_20201008_131.jpg',
 'V006_79_1_07_04_03_13_1_5950b_20201008_129.jpg',
 'V006_79_1_07_04_03_13_1_5950b_20201008_136.jpg',
 'V006_79_1_07_04_03_13_1_595

In [19]:
## 흰가루병이 아닌 이미지 모두 삭제 
## 정상에서는 사용하지 말것
## 질병코드(index 3): 고추 02, 애호박 08, 오이 12, 호박 20
## 작물코드(index 4): 고추 01, 애호박 04, 오이 06, 호박 10

def disease_code(datas): 
    new_datas = []
    for data in datas:
        crops = data.split('_')[4]
        disease = data.split('_')[3]
        
        if crops == '01' and disease == '02': # 고추흰가루병
            new_datas.append(data) 
            
        elif crops == '04' and disease == '08': # 애호박 흰가루병
            new_datas.append(data) 
                
        elif crops == '06' and disease == '12': # 오이 흰가루병
            new_datas.append(data)  
                
        elif crops == '10' and disease == '20':  # 호박 흰가루병
            new_datas.append(data) 
    return new_datas

In [20]:
data = disease_code(leaf)
data

['V006_79_1_08_04_03_13_1_1044bb_20201008_12.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201008_35.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201008_3.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201008_34.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201008_2.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201008_27.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201008_29.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201008_45.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201001_4.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201008_30.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201008_28.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201015_511.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201008_5.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201008_54.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201015_512.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201008_9.jpg',
 'V006_79_1_08_04_03_13_1_1044bb_20201015_518.jpg',
 'V006_79_1_08_04_03_13_1_3033t_20201008_392.jpg',
 'V006_79_1_08_04_03_13_1_3033t_20201008_390.jpg',
 'V006_79_1_08_04_03_13_1_3033t_2

In [21]:
len(data) # train_datas : len = 484

484

In [ ]:
# 현재 작업 위치
print(os.getcwd())

In [ ]:
root_path = '/content/drive/MyDrive/Colab/img' ## root path
os.mkdir(root_path)

train_dir = os.path.join(root_path,'train') 
os.mkdir(train_dir)

valid_dir = os.path.join(root_path,'valid') 
os.mkdir(valid_dir)

test_dir = os.path.join(root_path,'test') 
os.mkdir(test_dir)

In [ ]:
# # 일반 폴더나 파일을 tar 타입으로 압축할 경우
# tar -cvf 생성될파일명 압축할대상

# # tar 타입을 압축 해제할 경우
# tar -xvf 압축을해제할파일명

In [ ]:
# # 파일 업로드
# from google.colab import files

# # 파일이름 dental_image.tar.gz
# uploaded = files.upload()

# !ls -al 

In [ ]:
# # 기존 폴더 있으면 삭제
# !rm -rf dental_image

# # 압축 파일을 풀기
# !tar xvfz dental_image.tar.gz

# # 파일 내용 보기
# !ls -al

## 모델

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers, initializers, regularizers, metrics
from tensorflow.keras import layers, models

In [ ]:
# 사전학습된 VGG16 모델 불러오기
base_model = VGG16(include_top = False, weights = 'imagenet', input_shape=(224, 224, 3))

# weights : 가중치 모델 지정 ( None : 초기화된 가중치, 'imagenet' : 사전 학습된 가중치 )
# include_top : 네트워크의 최상위 완전 연결 분류기(fc)를 포함할지 안할지 지정
#               별도로 식물을 구분하는 완전 연결 층을 추가하려고 하므로, 이를 포함하지 않음 (False)
# input_tensor : 입력 텐서 크기 ( Input(shape = (넓이, 높이, 채널)) - 지정하지 않으면 어떤 크기의 입력도 처리 가능
# input_shape : 앞에 2개 숫자는 가로, 세로를 가리키며, 마지막 숫자인 3은 R,G,B 3개를 의미한다.
base_model.summary()   # 최종 특성 맵의 크기 (7, 7, 512)

58900480/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
# 모델 Layer 데이터화
layer_dict = dict([(layer.name, layer) for layer in base_model.layers])

# Layer 추가
x = layer_dict['block5_pool'].output

x = Conv2D(filters = 64, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(5, activation='softmax')(x)   # Softmax 함수로 5개 분류

# new model 정의
new_model = Model(inputs=base_model.input, outputs=x)

In [ ]:
# Convolution Layer를 학습되지 않도록 고정
# 새롭게 추가된 층 이외의 VGG16의 19번째 레이어까지 학습하지 않고 고정
for layer in base_model.layers[:19]:
    layer.trainable = False

new_model.summary()

# 모델 컴파일
new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# 사용하는 이유 : 별도의 파일을 안 만들고, 위처럼 폴더의 이름 자체를 label 하는 방법을 이제부터 할 예정이다. 즉 train안에 폴더 3개를 보고 각각 분류가 됐다는 걸 알 수 있다. 
#                 label의 정보를 주는 방법 중 하나라고 생각하면 될 것이다. 이 폴더를 학습을 시키면 labeling 없이 학습이 된다.

# train_dir, validation_dir은 위에서 정의하기
batch_size = 20
image_size = 224

# 학습에 사용될 이미지 데이터 생성기
train_datagen = ImageDataGenerator(
      rotation_range=180, # 회전 쵀대 20도
      width_shift_range=0.2, # 좌우 이동
      height_shift_range=0.2, # 상하 이동
      horizontal_flip=True, # 좌우 반전
      vertical_flip=True, # 상하 반전
      )
 
# 검증에 사용될 이미지 데이터 생성기
validation_datagen = ImageDataGenerator()
 

# 학습에 사용될 데이터 생성기  
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

# 검증에 사용될 데이터 생성기
validation_generator = validation_datagen.flow_from_directory(
        valid_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

class_num=len(train_generator.class_indices)

custom_labels = list(validation_generator.class_indices.keys())

In [ ]:
steps_epoch = len(train_generator)//batch_size
print(steps_epoch)

valid_steps = len(validation_generator)//batch_size
print(valid_steps)

In [ ]:
# 모델 학습

# Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 50 batches)
# You may need to use the repeat() function when building your dataset.
# 오류나서 steps_per_epoch와 epochs를 곱한 것이 train_generator 값보다 작게 바꿈 (각각 96, 20)
# steps_per_epoch와 validation_steps를 위에서 계산

history = new_model.fit(train_generator,
                        steps_per_epoch = steps_epoch,
                        epochs = 20,
                        validation_data = validation_generator,
                        validation_steps= valid_steps)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size = (224, 224),
                                                  batch_size = 20,
                                                  class_mode = 'categorical')

test_loss, test_acc = new_model.evaluate_generator(test_generator,steps=48)
print(f'test loss: {test_loss}, test acc: {test_acc}')